In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
import datetime 
import matplotlib.pyplot as plt

In [2]:
# Input
start_date = datetime.datetime(2014,1,1)
#end_date = datetime.datetime(2021,12,1)
end_date = datetime.datetime.today()

# 10 Stocks of same sector are chosen as representative of that sector
# tickers = ['BAJAJ-AUTO.NS','TCS.NS','INFY.NS',]
# tickers = ['HDFCBANK.NS','ICICIBANK.NS','KOTAKBANK.NS','SBIN.NS','AXISBANK.NS']
tickers = ['TATAPOWER.NS','RPOWER.NS','ADANIPOWER.NS']
data = pdr.get_data_yahoo(tickers,start_date,end_date)

In [3]:
data.tail()

Attributes    Adj Close                                 Close             \
Symbols    TATAPOWER.NS  RPOWER.NS ADANIPOWER.NS TATAPOWER.NS  RPOWER.NS   
Date                                                                       
2022-09-05   236.449997  23.299999    389.850006   236.449997  23.299999   
2022-09-06   247.350006  21.950001    407.750000   247.350006  21.950001   
2022-09-07   246.949997  21.650000    406.299988   246.949997  21.650000   
2022-09-08   244.899994  21.299999    401.899994   244.899994  21.299999   
2022-09-09   242.350006  19.200001    397.000000   242.350006  19.200001   

Attributes                       High                                   Low  \
Symbols    ADANIPOWER.NS TATAPOWER.NS  RPOWER.NS ADANIPOWER.NS TATAPOWER.NS   
Date                                                                          
2022-09-05    389.850006   238.399994  23.299999    407.000000   235.199997   
2022-09-06    407.750000   248.000000  25.000000    409.299988   237.199997   
2022-09-07    406.299988   251.149994  22.750000    415.750000   245.199997   
2022-09-08    401.899994   250.100006  22.350000    413.500000   243.600006   
2022-09-09    397.000000   247.000000  21.200001    406.850006   241.199997   

Attributes                                  Open                           \
Symbols     RPOWER.NS ADANIPOWER.NS TATAPOWER.NS  RPOWER.NS ADANIPOWER.NS   
Date                                                                        
2022-09-05  21.549999    385.549988   235.449997  21.549999    403.649994   
2022-09-06  21.400000    386.000000   237.649994  24.750000    392.000000   
2022-09-07  21.299999    401.250000   246.949997  21.750000    404.700012   
2022-09-08  21.150000    399.000000   249.550003  22.100000    413.000000   
2022-09-09  19.200001    395.000000   246.500000  21.200001    405.750000   

Attributes       Volume                             
Symbols    TATAPOWER.NS    RPOWER.NS ADANIPOWER.NS  
Date                                                
2022-09-05   10500643.0  153031552.0     3344426.0  
2022-09-06   61475311.0  151536084.0     3907288.0  
2022-09-07   30474615.0   64869145.0     2901689.0  
2022-09-08   14951235.0   48143250.0     2271073.0  
2022-09-09   12278497.0  115324167.0     2417344.0

In [4]:
temp_list = []
for i in tickers:
    temp_list.append(np.array([data['Adj Close'][i]]).T)

X = np.block([temp_list]).T

In [5]:
def DMD(X,r,dt,mf):
    '''
    Inputs:
        X = numpy.ndarray: Historical time series data with time along the columns and current time being the last column
        r = scalar: Number of main modes to consider
        dt = scaler: time step between each column
        mf = scalar: Number of future time steps you want to predict
    Outputs:
        X_DMD = mf+1 columns of predicted time series data with first column being the current time step
    '''
    X1,X2 = X[:,:-1],X[:,1:] # Last column of X is current price of the day close to closing
    U,s,Vh = np.linalg.svd(X1,full_matrices=False)
    r = min(r,np.shape(U)[1])
    Ur,Sr,Vr = U[:,:r],np.diag(s[:r]),Vh[:r,:].T
    A_T = Ur.T@X2@Vr@np.linalg.inv(Sr)
    d,Wr = np.linalg.eig(A_T)
    Phi = X2@Vr@np.linalg.inv(Sr)@Wr
    Lambda = np.diag(d)
    Omega = np.log(Lambda)/dt
    b = np.linalg.pinv(Phi)@X[:,-1:]
    
    t = np.arange(0,(mf+1)*dt,dt)
    t = t.reshape((t.shape[0],1,1))
    time_dynamics = np.exp(t*Omega)@b
    X_DMD = (Phi@time_dynamics).T.reshape((Phi.shape[0],t.shape[0]))
    return X_DMD
    

In [6]:
X_trial = X[:,-90:]
X_trial.shape

(3, 90)

In [10]:
test = 89
pred = 2
fut = 4
dmd = DMD(X_trial[:,test-pred:test],2,1,fut)
print('Current',X_trial[:,test-1])
print('Predicted',dmd[:,-1])
print('Actual',X_trial[:,test])
print('Error',np.sum((X_trial[:,test-1]-dmd[:,-1])**2)/len(tickers))

Current [244.8999939   21.29999924 401.8999939 ]
Predicted [235.09803489  20.44748097 385.81421454]
Actual [242.3500061   19.20000076 397.        ]
Error 118.5191618135464
